# Activity 2.2 - Transfer Learning

#### Objective(s):

This activity aims to introduce how to apply transfer learning

#### Intended Learning Outcomes (ILOs):
* Demonstrate how to build and train neural network
* Demonstrate how to apply transfer learning in neural network


#### Resources:
* Jupyter Notebook
* CIFAR-10 dataset

#### Procedures
Load the necessary libraries

In [ ]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Set the parameters

In [ ]:
now = datetime.datetime.now
batch_size = 128
num_classes = 5
epochs = 5
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

Set how the input data is loaded

In [ ]:
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

* Write a function to include all the training steps.
* Use the model, training set, test set and number of classes as function parameters


In [ ]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

Shuffle and split the data between train and test sets

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


Create two datasets
* one with digits below 5
* one with 5 and above

In [ ]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

* Define the feature layers that will used for transfer learning
* Freeze these layers during fine-tuning process

In [ ]:
feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

Define the classification layers

In [ ]:
classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

Create a model by combining the feature layers and classification layers

In [ ]:
model = Sequential(feature_layers + classification_layers)

Check the model summary

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

 Train the  model on the digits 5,6,7,8,9

In [ ]:
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 53s 227ms/step - loss: 1.6079 - accuracy: 0.2087 - val_loss: 1.5964 - val_accuracy: 0.2792
Epoch 2/5
230/230 [==============================] - 45s 198ms/step - loss: 1.5908 - accuracy: 0.2757 - val_loss: 1.5787 - val_accuracy: 0.4581
Epoch 3/5
230/230 [==============================] - 44s 191ms/step - loss: 1.5746 - accuracy: 0.3395 - val_loss: 1.5597 - val_accuracy: 0.5569
Epoch 4/5
230/230 [==============================] - 45s 197ms/step - loss: 1.5564 - accuracy: 0.4071 - val_loss: 1.5388 - val_accuracy: 0.6167
Epoch 5/5
230/230 [==============================] - 45s 194ms/step - loss: 1.5366 - accuracy: 0.4526 - val_loss: 1.5152 - val_accuracy: 0.6546
Training time: 0:04:23.264329
Test score: 1.5152373313903809
Test accuracy: 0.654597818851471


Freeze only the feature layers

In [ ]:
for l in feature_layers:
    l.trainable = False

Check again the summary and observe the parameters from the previous model

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

Train again the model using the 0 to 4 digits

In [ ]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 18s 74ms/step - loss: 1.5820 - accuracy: 0.2973 - val_loss: 1.5613 - val_accuracy: 0.4135
Epoch 2/5
240/240 [==============================] - 16s 67ms/step - loss: 1.5563 - accuracy: 0.3735 - val_loss: 1.5355 - val_accuracy: 0.5505
Epoch 3/5
240/240 [==============================] - 17s 70ms/step - loss: 1.5319 - accuracy: 0.4491 - val_loss: 1.5098 - val_accuracy: 0.6569
Epoch 4/5
240/240 [==============================] - 17s 71ms/step - loss: 1.5086 - accuracy: 0.5133 - val_loss: 1.4842 - val_accuracy: 0.7375
Epoch 5/5
240/240 [==============================] - 16s 67ms/step - loss: 1.4848 - accuracy: 0.5699 - val_loss: 1.4582 - val_accuracy: 0.7968
Training time: 0:02:22.761571
Test score: 1.458176851272583
Test accuracy: 0.7968476414680481


#### Supplementary Activity
Now write code to reverse this training process. That is, you will train on the digits 0-4, and then finetune only the last layers on the digits 5-9.

In [ ]:
for l in feature_layers:
    l.trainable = True

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

In [ ]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 46s 190ms/step - loss: 0.8514 - accuracy: 0.8083 - val_loss: 0.7534 - val_accuracy: 0.8834
Epoch 2/5
240/240 [==============================] - 45s 186ms/step - loss: 0.7953 - accuracy: 0.8304 - val_loss: 0.6939 - val_accuracy: 0.8986
Epoch 3/5
240/240 [==============================] - 44s 183ms/step - loss: 0.7418 - accuracy: 0.8441 - val_loss: 0.6370 - val_accuracy: 0.9082
Epoch 4/5
240/240 [==============================] - 45s 187ms/step - loss: 0.6968 - accuracy: 0.8489 - val_loss: 0.5845 - val_accuracy: 0.9117
Epoch 5/5
240/240 [==============================] - 46s 191ms/step - loss: 0.6486 - accuracy: 0.8552 - val_loss: 0.5352 - val_accuracy: 0.9175
Training time: 0:04:22.981225
Test score: 0.5351712107658386
Test accuracy: 0.9174937009811401


An observable difference between the one in the procedure and the one currently being trained, is that the validation accuracy has drastically increase. This could be due to the model already have data prior.

In [ ]:
for l in feature_layers[:-1]:
    l.trainable = False

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

In [ ]:
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 17s 71ms/step - loss: 1.1427 - accuracy: 0.5580 - val_loss: 1.0231 - val_accuracy: 0.6416
Epoch 2/5
230/230 [==============================] - 16s 71ms/step - loss: 1.0797 - accuracy: 0.5969 - val_loss: 0.9669 - val_accuracy: 0.6852
Epoch 3/5
230/230 [==============================] - 16s 69ms/step - loss: 1.0350 - accuracy: 0.6235 - val_loss: 0.9215 - val_accuracy: 0.7161
Epoch 4/5
230/230 [==============================] - 16s 68ms/step - loss: 0.9894 - accuracy: 0.6562 - val_loss: 0.8822 - val_accuracy: 0.7402
Epoch 5/5
230/230 [==============================] - 16s 71ms/step - loss: 0.9547 - accuracy: 0.6761 - val_loss: 0.8471 - val_accuracy: 0.7603
Training time: 0:01:22.704341
Test score: 0.8470866084098816
Test accuracy: 0.7603373527526855


When comparing the results of the supplementary and procedure training of digits 5-9 we can see that there is already a learned results. This helps visualize the "transfer" of previous knowledge. This method helps us in improving the general performance of our model.

#### Conclusion

Transfer learning takes advantage of previously trained layers/models and uses it on other data. This aids in the performance of the model, this is due to the model utilizing what it had previously learned and used it on the proceeding data. Overall this technique could be used when dealing with similar data as what was previously already learned by the model. This has also taught me of another way to improve the performance of the model.